In [6]:
from sklearn.datasets import load_iris
import pandas as pd
iris= load_iris()
df=pd.DataFrame(iris.data, columns=iris.feature_names)
df['flower'] =iris.target
df['flower']=df['flower'].apply(lambda x:iris.target_names[x])
df[47:53]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data,iris.target, test_size=0.2,)

In [12]:
from sklearn.svm import SVC
model = SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train, y_train)
model.score(X_test, y_test)

1.0

In [17]:
from sklearn.model_selection import cross_val_score
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [19]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [20]:
cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [23]:
import numpy as np
kernels=['rbf','linear']
c=[1,10,20]
avg_scores={}
for kval in kernels:
    for cval in c:
        cv_scores=cross_val_score(SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
        avg_scores[kval+'_'+str(cval)]=np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [ ]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(SVC(gamma='auto'),
    param_grid={'kernel':['rbf','linear'],
    'C':[1,10,20]},cv=5,return_train_score=True)
clf.fit(iris.data,iris.target)
clf.cv_results_


{'mean_fit_time': array([0.00287108, 0.00174766, 0.00132523, 0.00119128, 0.00246053,
        0.00135512]),
 'std_fit_time': array([0.00093557, 0.00056972, 0.00040152, 0.00037725, 0.0004034 ,
        0.00075637]),
 'mean_score_time': array([0.00261712, 0.00172515, 0.0012866 , 0.00148106, 0.00108504,
        0.00102644]),
 'std_score_time': array([0.00176991, 0.00044433, 0.00031063, 0.00060116, 0.00058274,
        0.00032152]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'split0_test_score': ar

In [30]:
df=pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.002871,0.000936,0.002617,0.001770,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1,0.983333,0.983333,0.991667,0.983333,0.975000,0.983333,0.005270
1,0.001748,0.000570,0.001725,0.000444,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1,0.975000,0.975000,0.991667,0.983333,0.983333,0.981667,0.006236
2,0.001325,0.000402,0.001287,0.000311,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1,0.975000,0.966667,0.991667,0.983333,0.966667,0.976667,0.009718
3,0.001191,0.000377,0.001481,0.000601,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4,0.966667,0.966667,0.991667,0.991667,0.975000,0.978333,0.011304
4,0.002461,0.000403,0.001085,0.000583,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5,0.966667,0.975000,0.991667,0.991667,0.975000,0.980000,0.010000
5,0.001355,0.000756,0.001026,0.000322,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6,0.975000,0.966667,0.991667,1.000000,0.975000,0.981667,0.012247


In [31]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [35]:
clf.best_score_


0.9800000000000001

In [34]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [39]:
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(SVC(gamma='auto'),
    param_distributions={'kernel':['rbf','linear'],
     'C':[1,10,20]},n_iter=2,cv=5,
     return_train_score=True)
rs.fit(iris.data,iris.target)
df=pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
df

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,10,linear,0.973333


In [40]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [41]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [44]:
import warnings; 
warnings.filterwarnings('ignore')
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df


,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
